# 0. Setup Paths

In [10]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH + '/annotations'
IMAGE_PATH = WORKSPACE_PATH + '/images'
MODEL_PATH = WORKSPACE_PATH + '/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH + '/pre-trained-models'
CONFIG_PATH = MODEL_PATH + '/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH + '/my_ssd_mobnet/'

# 1.Create Label map 

In [2]:
labels = [{'name' : 'LongPant', 'id': 1}, {'name': 'ShortPant', 'id': 2}]

In [3]:
labels

[{'name': 'LongPant', 'id': 1}, {'name': 'ShortPant', 'id': 2}]

In [11]:
with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [16]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


# 3. Download TF Model Pretrained Models from eTensorflow Model Zoo

In [17]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

Cloning into 'models'...
Updating files:  32% (821/2555)
Updating files:  33% (844/2555)
Updating files:  34% (869/2555)
Updating files:  35% (895/2555)
Updating files:  36% (920/2555)
Updating files:  37% (946/2555)
Updating files:  38% (971/2555)
Updating files:  39% (997/2555)
Updating files:  40% (1022/2555)
Updating files:  41% (1048/2555)
Updating files:  42% (1074/2555)
Updating files:  43% (1099/2555)
Updating files:  44% (1125/2555)
Updating files:  45% (1150/2555)
Updating files:  46% (1176/2555)
Updating files:  47% (1201/2555)
Updating files:  48% (1227/2555)
Updating files:  49% (1252/2555)
Updating files:  50% (1278/2555)
Updating files:  51% (1304/2555)
Updating files:  52% (1329/2555)
Updating files:  53% (1355/2555)
Updating files:  54% (1380/2555)
Updating files:  55% (1406/2555)
Updating files:  56% (1431/2555)
Updating files:  57% (1457/2555)
Updating files:  58% (1482/2555)
Updating files:  59% (1508/2555)
Updating files:  60% (1533/2555)
Updating files:  61% (1559

# 4. Copy Model Config to Training Folder

In [18]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'

In [26]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
!cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}

하위 디렉터리 또는 파일 Tensorflow\workspace\models\my_ssd_mobnet이(가) 이미 있습니다.
'cp'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [21]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [23]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'


In [27]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [28]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 4e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.01
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.997
         scale: true
         epsilon: 0.001
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
   